# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [10]:
#new
context_user = context.copy()
print(return_CCRMSQL(""""Find all employees whose salary is between 50,000 and 100,000 USD."
""", context_user))

```sql
SELECT * 
FROM employees
WHERE salary BETWEEN 50000 AND 100000;
```


In [11]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Retrieve the names of employees who studied at 'Harvard University'.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.Institution = 'Harvard University';
```

This SQL query retrieves the names of employees who studied at 'Harvard University' by joining the employees table with the studies table on the ID_usr field and filtering the results based on the Institution being 'Harvard University'.


In [12]:
#new
print(return_CCRMSQL("Calculate the average salary of employees in each department.", context_user))

```sql
SELECT department, AVG(salary) AS average_salary
FROM employees
GROUP BY department;
```


In [13]:
#old
print(return_CCRMSQL("Retrieve the names of employees who studied at 'Harvard University'.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.Institution = 'Harvard University';
```

This SQL query retrieves the names of employees who studied at 'Harvard University' by joining the employees table with the studies table on the ID_usr field and filtering the results to only include rows where the Institution is 'Harvard University'.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In this exercise, GPT showed consistent results in identifying relevant tables for SQL queries. However, there were certain limitations:

Lack of Depth in Responses: While the model correctly identified the necessary tables, it didn’t provide detailed explanations or relationships between the tables.
No Hallucinations: GPT did not fabricate non-existent tables, but in some cases, it overly simplified its answers by only listing tables without addressing subtle variations in the query.
Handling Variations: Different phrasings of the same query yielded identical responses, indicating the model struggled to capture nuanced differences in user intent (e.g., between retrieving data and performing calculations).
What did you learn?
Importance of Clear Prompts: Structured and explicit prompts play a critical role in guiding the model to provide accurate and relevant responses.
Model Strengths: GPT excels at mapping user queries to table descriptions when the context is well-defined, demonstrating its ability to process structured information effectively.
Limitations of General Models: GPT struggles to generate actionable outputs for complex tasks, such as writing detailed SQL queries or inferring relationships across tables without additional guidance.
Iterative Testing: Experimenting with different use cases revealed the model's consistent ability to select tables, but also highlighted areas where more detailed prompts or fine-tuning would improve performance.
Overall, GPT performs well for table selection in NL2SQL tasks but requires additional instructions or customization for generating complex queries or understanding nuanced variations.